# Business Intelligence Analyst Workbook

Answer the Following Questions in the Jupyter Notebook
Please do not use any Python packages to solve these problems. (Packages to import data and run SQL is fine).
If you have any questions, reach out to zach.fifelski@prizepicks.com
You can print your results or just write the SQL query if you're having issues with the notebook.
If you're not familiar with Postgres you are more than welcome to write in your prefered syntax

In [1]:
## NAME HERE
## Aaron Khan

In [7]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'left')
pd.options.mode.chained_assignment = None 



conn = psycopg2.connect(
   database="postgres",
    user='postgres',
    password='thepassword',
    host='localhost',
    port= '5432'
)
 
conn.autocommit = True
 
# Creating a cursor object
cursor = conn.cursor()
 
# query to create a database
sql = ''' CREATE database BIAnalystDB ''';
 
# executing above query
cursor.execute(sql)
print("Database has been created successfully !!");
 
# Closing the connection
conn.close()



  

  
conn = psycopg2.connect(database="bianalystdb",
                        user='postgres', password='thepassword', 
                        host='127.0.0.1', port='5432'
)
  
conn.autocommit = True
cursor = conn.cursor()

sql1 = '''CREATE TABLE entry_data(
            entry_date Date,
            league char(10),
            player char(30),
            user_id char(50),
            entry_amount float);'''

cursor.execute(sql1)

sql4 = '''CREATE TABLE signup_data(
            reg_date Date,
            user_id char(100));'''
  
cursor.execute(sql4)
  
sql2 = '''COPY entry_data(entry_date,league,player,user_id,entry_amount)
FROM 'C:\Business Analyst Assessment\Business_Analyst_Assessment_\SA_entry_data.csv'
DELIMITER ','
CSV HEADER;'''

cursor.execute(sql2)

sql5 = '''COPY signup_data(user_id,reg_date)
FROM 'C:\Business Analyst Assessment\Business_Analyst_Assessment_\SA_signup_data.csv'
DELIMITER ','
CSV HEADER;'''

cursor.execute(sql5)
    
sql3 = '''select * from entry_data;'''
cursor.execute(sql3)
for i in cursor.fetchall():
    print(i)
  
# conn.commit()
conn.close()



DuplicateDatabase: database "bianalystdb" already exists


## 1.

Treat SA_entry_data.csv and SA_signup_data.csv as two tables in a SQL DB named entry_data and signup_data respectively. Write sql queries to answer the following 4 questions

a) What is the total entry amount for all members who signed up in September, 2020?

In [8]:

conn = psycopg2.connect(database="bianalystdb",
                        user='postgres', password='thepassword', 
                        host='127.0.0.1', port='5432'
)
  
conn.autocommit = True
cursor = conn.cursor()

sql ='''SELECT 
        sum(entry_amount) AS total_entry 
        FROM entry_data ed 
        JOIN signup_data sd ON sd.user_id = ed.user_id 
        WHERE EXTRACT(MONTH FROM reg_date) = 9 AND EXTRACT(YEAR FROM reg_date) = 2020'''

df = pd.read_sql(sql, con = conn)
print(df)
  
# conn.commit()
conn.close()

   total_entry
0  1495076.5  


b) What is the average entry amount per member for each day of the week (Monday-Sunday) in the month of September, 2020?

In [9]:
 
conn = psycopg2.connect(database="bianalystdb",
                        user='postgres', password='thepassword', 
                        host='127.0.0.1', port='5432'
)
  
conn.autocommit = True
cursor = conn.cursor()

sql ='''SELECT
        AVG(entry_amount) AS AvgEntry,
        to_char(entry_date, 'Day') AS DayofWeek 
        FROM entry_data 
        WHERE to_char(entry_date, 'Month') = 'September'
        GROUP BY DayofWeek 
        ORDER BY AvgEntry DESC
        '''

df = pd.read_sql(sql, con = conn)
print(df)
  
# conn.commit()
conn.close()

   avgentry   dayofweek 
0  315.788505  Saturday 
1  315.213912  Tuesday  
2  313.205777  Wednesday
3  312.961514  Friday   
4  312.127630  Thursday 
5  310.576259  Monday   
6  308.977429  Sunday   


c) Create an ordered list of the top 5 most popular players (by total entry amount) and their league in the month of September 2020

In [10]:
 
conn = psycopg2.connect(database="bianalystdb",
                        user='postgres', password='thepassword', 
                        host='127.0.0.1', port='5432'
)
  
conn.autocommit = True
cursor = conn.cursor()

sql ='''SELECT 
        SUM(entry_amount) AS total_entry,player, 
        League
        FROM entry_data
        WHERE to_char(entry_date, 'Month') = 'September'
        GROUP BY Player,league
        ORDER BY total_entry DESC
        LIMIT 5 '''

df = pd.read_sql(sql, con = conn)
print(df)
  
# conn.commit()
conn.close()

   total_entry player                          league               
0  165348.0     Jamal Murray                    NBA                 
1  163722.0     Nikola Jokic                    NBA                 
2  162286.5     LeBron James                    NBA                 
3  159838.5     Jayson Tatum                    NBA                 
4  151018.0     Jimmy Butler                    NBA                 


d) Determine each user's contribution (by percentage) to the total entry amount for each player.

The final output should include 3 columns: player_name, user_id, and share. 

Note: You do not need to format the final column with a percentage sign

Example Output:

| player_name | user_id | share |
| --- | --- | --- |
| Lebron James | Fedor | 3.500000 |
| Lebron James | Rachel | 96.500000 |
| Michael Jordan | Fedor | 1.404454 |
| Michael Jordan | Rachel | 98.595546 |

In [11]:
 
conn = psycopg2.connect(database="bianalystdb",
                        user='postgres', password='thepassword', 
                        host='127.0.0.1', port='5432'
)
  
conn.autocommit = True
cursor = conn.cursor()

sql ='''SELECT 
        player AS player_name,
        user_id,
        (entry_amount / sum(entry_amount) over (PARTITION BY player)) * 100 AS share
        FROM entry_data;
        '''


df = pd.read_sql(sql, con = conn)
print(df)
  
# conn.commit()
conn.close()




      player_name                     user_id                                                                                                share   
0      A.J. Brown                      339accbe-406e-472d-b57f-c8fde8e68852                                                                  4.513211
1      A.J. Brown                      fe1ada22-7eef-4be0-8e45-5aac81a5e97e                                                                  4.703642
2      A.J. Brown                      ef58a067-55a6-43a5-95f9-fc00db60a3c4                                                                  2.913592
3      A.J. Brown                      ef58a067-55a6-43a5-95f9-fc00db60a3c4                                                                  3.370626
4      A.J. Brown                      e7abe9a1-8534-47da-af7b-04b3b1dad18c                                                                  2.556534
...                               ...                                                               

# This is the end of the SQL/Python portion